In [2]:
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
import requests

In [3]:
from langchain_core.tools import StructuredTool
from pydantic import BaseModel, Field
class MultiplyInput(BaseModel):
    a: int = Field(required=True, description="The first number to add")
    b: int = Field(required=True, description="The second number to add")
def multiply_func(a: int, b: int) -> int:
    return a * b
multiply_tool = StructuredTool.from_function(
    func=multiply_func,
    name="multiply",
    description="Multiply two numbers",
    args_schema=MultiplyInput
)


result = multiply_tool.invoke({'a':3, 'b':3})

print(result)
print(multiply_tool.name)
print(multiply_tool.description)
print(multiply_tool.args)

9
multiply
Multiply two numbers
{'a': {'description': 'The first number to add', 'title': 'A', 'type': 'integer'}, 'b': {'description': 'The second number to add', 'title': 'B', 'type': 'integer'}}


C:\Users\dell\AppData\Local\Temp\ipykernel_10896\1022193022.py:4: PydanticDeprecatedSince20: Using extra keyword arguments on `Field` is deprecated and will be removed. Use `json_schema_extra` instead. (Extra keys: 'required'). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  a: int = Field(required=True, description="The first number to add")
C:\Users\dell\AppData\Local\Temp\ipykernel_10896\1022193022.py:5: PydanticDeprecatedSince20: Using extra keyword arguments on `Field` is deprecated and will be removed. Use `json_schema_extra` instead. (Extra keys: 'required'). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  b: int = Field(required=True, description="The second number to add")


# tool binding

In [4]:
llm = ChatOpenAI()

llm.invoke('hi')

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 8, 'total_tokens': 17, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-D2kjwpgaMaxh3xn73ZGVL9unSPJik', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019c0152-3d54-7172-9c4d-c0fb8be9eb27-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 8, 'output_tokens': 9, 'total_tokens': 17, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

Tool Binding

In [5]:
llm_with_tools = llm.bind_tools([multiply_tool])

In [6]:
llm_with_tools.invoke('Hi how are you')

AIMessage(content="I'm just a computer program, so I don't have feelings, but I'm here to help you! How can I assist you today?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 30, 'prompt_tokens': 64, 'total_tokens': 94, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-D2kjxrRlv53AVbyzC8Kl8qt2bdApB', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019c0152-4de8-7001-9242-96a91757a8bb-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 64, 'output_tokens': 30, 'total_tokens': 94, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [17]:
query = HumanMessage('can you multiply 3 with 1000')
messages = [query]

Tool Calling

In [18]:
result = llm_with_tools.invoke(messages)

In [19]:
messages.append(result)

In [20]:
result.tool_calls[0]

{'name': 'multiply',
 'args': {'a': 3, 'b': 1000},
 'id': 'call_btghuhalLWKf7U8FQUsUDrQe',
 'type': 'tool_call'}

Tool Execution

In [21]:
tool_result = multiply_tool.invoke(result.tool_calls[0])
tool_result

ToolMessage(content='3000', name='multiply', tool_call_id='call_btghuhalLWKf7U8FQUsUDrQe')

In [22]:
messages.append(tool_result)

In [24]:
messages

[HumanMessage(content='can you multiply 3 with 1000', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 69, 'total_tokens': 87, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-D2kxISQ8PQGiszjpKtYGY6nsZESzr', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019c015e-e9f2-7ea3-babb-e9fc9829273a-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 1000}, 'id': 'call_btghuhalLWKf7U8FQUsUDrQe', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 69, 'output_tokens': 18, 'total_tokens': 87, 'input_token_details': {'audio': 0, 'cac

In [25]:
llm_with_tools.invoke(messages).content

'The result of multiplying 3 with 1000 is 3000.'